In [1]:
import tensorflow as tf
import keras
import time
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator

import qkeras
from qkeras import QActivation
from qkeras import QDense, QConv2DBatchnorm

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l1
from tensorflow.keras.models import Model

from callbacks import all_callbacks

2023-06-28 18:34:10.535008: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

cifar10_classes=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [3]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, MaxPooling2D, Flatten, Activation
from qkeras import QActivation, QDense, QConv2D
from qkeras.quantizers import quantized_bits, quantized_relu
from tensorflow.keras.regularizers import l1, l2

# 输入层
input_layer = Input(shape=(32, 32, 3))

# 第一层卷积层
x = QConv2D(
    64,
    kernel_size=(3, 3),
    strides=(4, 4),
    padding='valid',
    name='conv1',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l2(0.0001)
)(input_layer)
x = QActivation(activation=quantized_relu(6), name='relu1')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid', name='pool1')(x)

# 第二层卷积层
x = QConv2D(
    4,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='same',
    name='conv2',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l2(0.0001)
)(x)
x = QActivation(activation=quantized_relu(6), name='relu2')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='pool2')(x)

# 第三层卷积层
x = QConv2D(
    64,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='same',
    name='conv3',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l2(0.0001)
)(x)
x = QActivation(activation=quantized_relu(6), name='relu3')(x)

# 第四层卷积层
x = QConv2D(
    8,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='same',
    name='conv4',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l2(0.0001)
)(x)
x = QActivation(activation=quantized_relu(6), name='relu4')(x)

# 第五层卷积层
x = QConv2D(
    32,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding='same',
    name='conv5',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
)(x)
x = QActivation(activation=quantized_relu(6), name='relu5')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='pool5')(x)

# 全连接层
x = Flatten()(x)
x = QDense(
    64,
    name='fc1',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l2(0.0001)
)(x)
x = QActivation(activation=quantized_relu(6), name='relu6')(x)

x = QDense(
    32,
    name='fc2',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l2(0.0001)
)(x)
x = QActivation(activation=quantized_relu(6), name='relu7')(x)

x = QDense(
    10,
    name='output',
    kernel_quantizer=quantized_bits(6, 0, alpha=1),
    bias_quantizer=quantized_bits(6, 0, alpha=1),
    kernel_initializer='lecun_uniform',
    kernel_regularizer=l1(0.0001)
)(x)
output_layer = Activation(activation='softmax', name='softmax')(x)

# 构建模型
model = Model(inputs=input_layer, outputs=output_layer)

# 打印模型概况
model.summary()


2023-06-28 18:34:12.467626: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-06-28 18:34:12.518571: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-28 18:34:12.518887: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.74GiB deviceMemoryBandwidth: 573.69GiB/s
2023-06-28 18:34:12.518904: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-06-28 18:34:12.521124: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-06-28 18:34:12.521153: I tensorflow/stream_executor

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv1 (QConv2D)              (None, 8, 8, 64)          1792      
_________________________________________________________________
relu1 (QActivation)          (None, 8, 8, 64)          0         
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 3, 3, 64)          0         
_________________________________________________________________
conv2 (QConv2D)              (None, 3, 3, 4)           2308      
_________________________________________________________________
relu2 (QActivation)          (None, 3, 3, 4)           0         
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 2, 2, 4)           0     

In [4]:
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning

pruning_params = {"pruning_schedule": pruning_schedule.ConstantSparsity(0.75, begin_step=2000, frequency=100)}
model = prune.prune_low_magnitude(model, **pruning_params)

In [5]:
train = True
if train:
    adam = Adam(lr=0.001)
    model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
    
    callbacks = all_callbacks(
        stop_patience=1000,
        lr_factor=0.5,
        lr_patience=10,
        lr_epsilon=0.000001,
        lr_cooldown=2,
        lr_minimum=0.0000001,
        outputDir='AlexNet_Weights',
    )
    callbacks.callbacks.append(pruning_callbacks.UpdatePruningStep())
    
    model.fit(
        x_train,
        y_train,
        batch_size=1024,
        epochs=5000,
        validation_split=0.25,
        shuffle=True,
        callbacks=callbacks.callbacks,
    )
    # Save the model again but with the pruning 'stripped' to use the regular layer types
    model = strip_pruning(model)
    model.save('AlexNet_Weights/AlexNet_check_best_model.h5')
else:
    from tensorflow.keras.models import load_model
    from qkeras.utils import _add_supported_quantized_objects

    co = {}
    _add_supported_quantized_objects(co)
    model = load_model('AlexNet_Weights/AlexNet_check_best_model.h5', custom_objects=co)

/home/lab-shi/anaconda3/lib/python3.9/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
2023-06-28 18:34:16.074134: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2023-06-28 18:34:16.074151: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2023-06-28 18:34:16.074177: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1611] Profiler found 1 GPUs
2023-06-28 18:34:16.074642: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64::/usr/local/hdf5/lib
2023-06-28 18:34:16.074708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot 

Epoch 1/5000
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


2023-06-28 18:34:21.076004: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:808] layout failed: Invalid argument: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/model/prune_low_magnitude_relu5/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-06-28 18:34:21.646691: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-06-28 18:34:22.096858: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8101
2023-06-28 18:34:22.631046: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-06-28 18:34:23.038124: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


13/37 [=========>....................] - ETA: 0s - loss: 2.3458 - accuracy: 0.1197

2023-06-28 18:34:23.464484: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2023-06-28 18:34:23.464506: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2023-06-28 18:34:23.464567: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1661] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2023-06-28 18:34:23.479604: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2023-06-28 18:34:23.479673: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1752] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2023-06-28 18:34:23.485322: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 0 callback api events and 0 activity events. 
2023-06-28 18:34:23.488488: I tensorflow

...


Epoch 00974: saving model to AlexNet_Weights/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 975/5000
37/37 [==============================] - 1s 14ms/step - loss: 1.7949 - accuracy: 0.3106 - val_loss: 1.8122 - val_accuracy: 0.3050

***callbacks***
saving losses to AlexNet_Weights/losses.log

Epoch 00975: val_loss did not improve from 1.58687

Epoch 00975: val_loss did not improve from 1.58687

Epoch 00975: saving model to AlexNet_Weights/KERAS_check_model_last.h5

Epoch 00975: saving model to AlexNet_Weights/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 976/5000
37/37 [==============================] - 1s 14ms/step - loss: 1.7960 - accuracy: 0.3091 - val_loss: 1.8139 - val_accuracy: 0.3055

***callbacks***
saving losses to AlexNet_Weights/losses.log

Epoch 00976: val_loss did not improve from 1.58687

Epoch 00976: val_loss did not improve from 1.58687

Epoch 00976: saving model to AlexNet_Weights/KERAS_check_model_last.h5

Epoch 00976: saving model to Al


Epoch 00991: saving model to AlexNet_Weights/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 992/5000
37/37 [==============================] - 1s 14ms/step - loss: 1.7958 - accuracy: 0.3113 - val_loss: 1.8133 - val_accuracy: 0.3050

***callbacks***
saving losses to AlexNet_Weights/losses.log

Epoch 00992: val_loss did not improve from 1.58687

Epoch 00992: val_loss did not improve from 1.58687

Epoch 00992: saving model to AlexNet_Weights/KERAS_check_model_last.h5

Epoch 00992: saving model to AlexNet_Weights/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 993/5000
37/37 [==============================] - 1s 14ms/step - loss: 1.7960 - accuracy: 0.3099 - val_loss: 1.8135 - val_accuracy: 0.3065

***callbacks***
saving losses to AlexNet_Weights/losses.log

Epoch 00993: val_loss did not improve from 1.58687

Epoch 00993: val_loss did not improve from 1.58687

Epoch 00993: saving model to AlexNet_Weights/KERAS_check_model_last.h5

Epoch 00993: saving model to Al


Epoch 01008: saving model to AlexNet_Weights/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 1009/5000
37/37 [==============================] - 1s 14ms/step - loss: 1.7953 - accuracy: 0.3107 - val_loss: 1.8124 - val_accuracy: 0.3030

***callbacks***
saving losses to AlexNet_Weights/losses.log

Epoch 01009: val_loss did not improve from 1.58687

Epoch 01009: val_loss did not improve from 1.58687

Epoch 01009: saving model to AlexNet_Weights/KERAS_check_model_last.h5

Epoch 01009: saving model to AlexNet_Weights/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 1010/5000
37/37 [==============================] - 1s 14ms/step - loss: 1.7959 - accuracy: 0.3102 - val_loss: 1.8127 - val_accuracy: 0.3046

***callbacks***
saving losses to AlexNet_Weights/losses.log

Epoch 01010: val_loss did not improve from 1.58687

Epoch 01010: val_loss did not improve from 1.58687

Epoch 01010: saving model to AlexNet_Weights/KERAS_check_model_last.h5

Epoch 01010: saving model to 


Epoch 01025: saving model to AlexNet_Weights/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 1026/5000
37/37 [==============================] - 1s 14ms/step - loss: 1.7952 - accuracy: 0.3115 - val_loss: 1.8122 - val_accuracy: 0.3054

***callbacks***
saving losses to AlexNet_Weights/losses.log

Epoch 01026: val_loss did not improve from 1.58687

Epoch 01026: val_loss did not improve from 1.58687

Epoch 01026: saving model to AlexNet_Weights/KERAS_check_model_last.h5

Epoch 01026: saving model to AlexNet_Weights/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 1027/5000
37/37 [==============================] - 1s 14ms/step - loss: 1.7951 - accuracy: 0.3115 - val_loss: 1.8126 - val_accuracy: 0.3052

***callbacks***
saving losses to AlexNet_Weights/losses.log

Epoch 01027: val_loss did not improve from 1.58687

Epoch 01027: val_loss did not improve from 1.58687

Epoch 01027: saving model to AlexNet_Weights/KERAS_check_model_last.h5

Epoch 01027: saving model to 


Epoch 01042: saving model to AlexNet_Weights/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 1043/5000
37/37 [==============================] - 1s 14ms/step - loss: 1.7959 - accuracy: 0.3120 - val_loss: 1.8163 - val_accuracy: 0.3035

***callbacks***
saving losses to AlexNet_Weights/losses.log

Epoch 01043: val_loss did not improve from 1.58687

Epoch 01043: val_loss did not improve from 1.58687

Epoch 01043: saving model to AlexNet_Weights/KERAS_check_model_last.h5

Epoch 01043: saving model to AlexNet_Weights/KERAS_check_model_last_weights.h5

***callbacks end***

Epoch 1044/5000
37/37 [==============================] - 1s 14ms/step - loss: 1.7956 - accuracy: 0.3100 - val_loss: 1.8127 - val_accuracy: 0.3042

***callbacks***
saving losses to AlexNet_Weights/losses.log

Epoch 01044: val_loss did not improve from 1.58687

Epoch 01044: val_loss did not improve from 1.58687

Epoch 01044: saving model to AlexNet_Weights/KERAS_check_model_last.h5

Epoch 01044: saving model to 